<a href="https://colab.research.google.com/github/lupries/TAGN2/blob/master/scripts/FullModelTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lupries/TAGN2.git
!cd TAGN2 && git pull

Cloning into 'TAGN2'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 224 (delta 87), reused 137 (delta 29), pack-reused 0
Receiving objects: 100% (224/224), 8.67 MiB | 12.13 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Already up to date.


In [2]:
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-1.4.0-cp36-cp36m-linux_x86_64.whl size=3170365 sha256=8a49b014ad0f42109d49d3a6050ec3ec8be3f70ca424d2e6ad162b3b0f56d3f1
  Stored in directory: /root/.cache/pip/wheels/25/00/c4/1637b4b3003f29092f4fe2ad4b40dd10906269c1ac2dc82941
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.4.3-cp36-cp36m-linux_x86_64.whl size=3966625 sha256=3da4724d1c98208f7f44d21c02f1a94b7a77e5e416f4f77ab9f094ab1b02d423
  Stored in directory: /root/.cache/pip/wheels/02/66/2b/befece01c2516f9fb3e7b4d150bb2b871221c73657c9cd7735
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.4.5-cp36-cp36m-linux_x86_64.whl size=16231819 sha256=addf11d636629caf875536123a0c48a0119f5a093f45de89e09f3690df4a4e1c
  Stored in directory: /root/.cache/pip/wheels/0a/26/7e/a6d6a80eae5ca39b92bc77773f36cf433d5085de18014382b1
Successfully built torch-cluster
     |████████████████████████████████| 133kB 7.8MB

In [0]:
import torchfrom torch.nn import functional as F
import torch.nn as nn

from TAGN2.models.graphnet import AGNN
from torchvision import models

class TAGNN(nn.Module):

    def __init__(self, loops):
        super(TAGNN, self).__init__()

        deeplab = models.segmentation.deeplabv3_resnet50(pretrained=False)
        self.backbone = deeplab.backbone
        self.graph = AGNN(loops=loops, channels=2048)
        self.readout = models.segmentation.deeplabv3.DeepLabHead(2048, num_classes=1)

    def forward(self, x):

        input_shape = x.shape[-2:]
        features = self.backbone(x)
        x = features['out']
        x = self.graph(x)
        x = self.readout(x)

        return F.interpolate(x, size=input_shape, mode='bilinear', align_corners=False)
    


In [25]:
model = TAGNN(loops=2)
model.cuda()

/content/TAGN2/models/convgru/convgru.py:22: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.reset_gate.weight)
/content/TAGN2/models/convgru/convgru.py:23: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.update_gate.weight)
/content/TAGN2/models/convgru/convgru.py:24: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.out_gate.weight)
/content/TAGN2/models/convgru/convgru.py:25: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.reset_gate.bias, 0.)
/content/TAGN2/models/convgru/convgru.py:26: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.update_gate.bias, 0.)
/content/TAGN2/models/convgru/convgru.py:27: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.out_gate.bias

TAGNN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequen

In [28]:
data = torch.ones(5,3,256,256).cuda()
print("data: " + str(data.shape))
output = model(data)
print("output: " + str(output.shape))

data: torch.Size([5, 3, 256, 256])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


output: torch.Size([5, 1, 256, 256])
